# Generasi Teks dengan RNN/LSTM menggunakan JAX dan Flax NNX

Notebook ini mendemonstrasikan cara membangun model bahasa sederhana untuk menghasilkan teks secara otomatis (text generation) menggunakan library JAX dan Flax NNX. Kita akan menggunakan kumpulan puisi Chairil Anwar sebagai data latih.

## Langkah 1: Persiapan Lingkungan dan Konfigurasi

Pertama, kita perlu mengatur path agar notebook dapat menemukan modul pendukung (`model_utils` dan `seq_processor`) serta mengonfigurasi JAX untuk berjalan di CPU guna menghindari masalah kompatibilitas pada beberapa perangkat Mac.

In [2]:
import sys, os
import jax
import jax.numpy as jnp
from flax import nnx
import optax
import numpy as np
from time import process_time

# Mengatur agar path mengarah ke root directory proyek JAX
script_dir = os.getcwd()
jax_dir = os.path.dirname(script_dir)
if jax_dir not in sys.path:
    sys.path.append(jax_dir)

import seq_processor as sp
import model_utils as mu

# # Paksa penggunaan CPU untuk stabilitas
# jax.config.update("jax_platform_name", "cpu")

# print("JAX Platform:", jax.lib.xla_bridge.get_backend().platform)

## Langkah 2: Memuat dan Memproses Data

Kita akan menggunakan file teks berisi puisi-puisi Chairil Anwar. Karakter-karakter dalam teks tersebut akan diubah menjadi representasi numerik (integer) agar dapat diproses oleh model neural network.

In [ ]:
data_dir = "../data"
data_path = os.path.join(data_dir, "chairilanwar.txt")

# Inisialisasi processor karakter
chproc = sp.CharProcessor(data_path)
data = jnp.array(chproc.encode(chproc.text), dtype=jnp.int32)

print(f"Total karakter: {len(data)}")
print(f"Ukuran vokabulari: {chproc.vocab_size}")

Length of text: 37970 characters

 !&()*+,-.0123456789:;?ABCDEFGHIJKLMNOPRSTUWYabcdefghijklmnoprstuvwxyzé–‘’“”…
78
Total karakter: 37970
Ukuran vokabulari: 78


## Langkah 3: Definisi Model dan Optimizer

Kita menggunakan arsitektur `SimpleBigram` yang didasarkan pada LSTM. Model ini telah dioptimasi menggunakan `jax.lax.scan` untuk menangani urutan karakter yang panjang secara efisien tanpa memperlambat proses kompilasi JIT.

In [4]:
seq_len = 256
n_embed = 384
n_hidden = 512
batch_size = 64

# Inisialisasi model
rngs = nnx.Rngs(1337)
model = mu.SimpleBigram(
    chproc.vocab_size,
    seq_len,
    n_embed,
    n_hidden,
    num_layers=1,
    rngs=rngs
)

# Inisialisasi optimizer dengan AdamW
optimizer = nnx.Optimizer(model, optax.adamw(3e-4), wrt=nnx.Param)

print("Model siap dilatih.")

Model siap dilatih.


## Langkah 4: Fungsi Pelatihan

Kita mendefinisikan `loss_fn` untuk menghitung error (cross-entropy) dan `train_step` yang dihiasi dengan `@nnx.jit` untuk mengeksekusi pelatihan secara cepat di XLA.

In [5]:
def loss_fn(model, xb, yb):
    logits = model(xb)
    B, T, C = logits.shape
    logits_flat = logits.reshape(B * T, C)
    targets_flat = yb.reshape(B * T)
    loss = optax.softmax_cross_entropy_with_integer_labels(logits_flat, targets_flat).mean()
    return loss

@nnx.jit
def train_step(model, optimizer, xb, yb):
    loss, grads = nnx.value_and_grad(loss_fn)(model, xb, yb)
    optimizer.update(model, grads)
    return loss

@nnx.jit(static_argnums=(2, 3, 4))
def estimate_loss(model, data, eval_iters=10, batch_size=32, seq_len=64, key=None):
    model.eval()
    losses = []
    for k in range(eval_iters):
        curr_key = jax.random.fold_in(key, k) if key is not None else jax.random.PRNGKey(k)
        xb, yb = sp.get_batch(data, batch_size=batch_size, block_size=seq_len, key=curr_key)
        loss = loss_fn(model, xb, yb)
        losses.append(loss)
    
    avg_loss = jnp.mean(jnp.array(losses))
    model.train()
    return avg_loss

## Langkah 5: Proses Pelatihan

Kita akan menjalankan iterasi pelatihan. Setiap interval tertentu, kita akan menghitung rata-rata loss dan mencoba menghasilkan beberapa teks awal untuk melihat perkembangan model.

In [6]:
max_iters = 100 # Dikurangi untuk demonstrasi cepat
eval_interval = 20
key = jax.random.PRNGKey(0)

print("Memulai pelatihan...")

for step in range(max_iters):
    key, subkey = jax.random.split(key)
    xb, yb = sp.get_batch(data, batch_size=batch_size, block_size=seq_len, key=subkey)

    start_t = process_time()
    loss = train_step(model, optimizer, xb, yb)
    elapsed_t = process_time() - start_t

    if step % eval_interval == 0 or step == max_iters - 1:
        key, subkey = jax.random.split(key)
        train_loss = estimate_loss(model, data, eval_iters=5, batch_size=batch_size, seq_len=seq_len, key=subkey)

        print(f"[Iter-{step+1}/{max_iters}] Loss: {train_loss:.4f} ({elapsed_t:.3f}s)")
        
        # Coba hasilkan teks singkat
        idx = jnp.zeros((1, 1), dtype=jnp.int32)
        pred_idx = model.generate(idx, 50, rngs=rngs)
        pred_str = chproc.decode(np.array(pred_idx[0]))
        print(f"--- Teks Tergenerasi ---\n{pred_str}\n------------------------")

Memulai pelatihan...
[Iter-1/100] Loss: 4.3447 (1.011s)
--- Teks Tergenerasi ---

()SrDPg6G27fWfjhp1n.A’lRKv*;dn?5 SEcf*P6LHO;?EKcjB
------------------------
[Iter-21/100] Loss: 3.2492 (0.006s)
--- Teks Tergenerasi ---

-NiY2’y5
!FiB eaakk0nnln, tB
jaL*tPhdpuuvi
nueti i
------------------------
[Iter-41/100] Loss: 3.1409 (0.005s)
--- Teks Tergenerasi ---

.c;Sls,+pPxp alodpjmdEeedbi: u AgbrbiseB a  rdrniN
------------------------
[Iter-61/100] Loss: 3.0425 (0.004s)
--- Teks Tergenerasi ---

z…uI–4PhA ijp
ra K d Klhu eda :riaa
a
 i
dDseii  a
------------------------
[Iter-81/100] Loss: 2.8326 (0.003s)
--- Teks Tergenerasi ---

mjYvzlNbkm regatnmbkK 4ami
ulanaearsa
i  Skaldkdli
------------------------
[Iter-100/100] Loss: 2.5986 (0.003s)
--- Teks Tergenerasi ---

IbHUdAld iamnna
hitadu
 jabAc
K
mAtan.e menyantgaa
------------------------


## Langkah 6: Generasi Teks Akhir

Setelah model dilatih, kita bisa menghasilkan teks yang lebih panjang.

In [7]:
print("Generating long text samples...")
idx = jnp.zeros((1, 1), dtype=jnp.int32)
pred_idx = model.generate(idx, 200, rngs=rngs)
pred_str = chproc.decode(np.array(pred_idx[0]))
print(f"Final Generated text:\n\n{pred_str}")

Generating long text samples...
Final Generated text:


yL.sk2 –
rer
sH kan temakupadDa,Ta tahuMb 1Albudan bai darhi
 meatukaka ippamean asRtu aenp,  etNu tatu. iiw d lan dak sendi auiuu 1enbasinlan gadi 
ahgiJus aeda btua L?denip
hbuu p nunga
garen yehu.a
